---

**Load essential libraries**

---

In [ ]:
import pandas as pd
import numpy as np
import sys
from scipy import linalg
import matplotlib.pyplot as plt
import matplotlib.cm as cm
plt.style.use('dark_background')
%matplotlib inline
plt.rcParams['figure.figsize'] = (4.0, 4.0) # set default size of plots

from sklearn.preprocessing import StandardScaler, RobustScaler

from scipy.sparse import random
from scipy import stats

---

Mount Google drive folder if running in Colab

---

In [ ]:
## Mount Google drive folder if running in Colab
if('google.colab' in sys.modules):
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    # Change path below starting from /content/drive/MyDrive/Colab Notebooks/
    # depending on how data is organized inside your Colab Notebooks folder in
    # Google Drive
    DIR = '/content/drive/MyDrive/Colab Notebooks/MAHE/MSIS Coursework/OddSem2025MAHE'
    DATA_DIR = DIR+'/Data/'
else:
    DATA_DIR = 'Data/'

---

Load ECG data

---

In [ ]:
# Load ECG data
FILE = DATA_DIR + 'ecg_missing.txt'
df_ecg = pd.read_csv(FILE, header = None, names = ['ecgvalues'])
print(df_ecg.head())
print(df_ecg['ecgvalues'].dtypes)
df_ecg['ecgvalues'] = df_ecg['ecgvalues'].astype(float)
print(df_ecg['ecgvalues'].dtypes)

---

Check if there are any missing values

---

In [ ]:
# Check if there are any missing values
?

---

Component plot of the ecg signal

---

In [ ]:
# Component plot of the ecg signal
fig, ax = plt.subplots(1, 1)
component_index = range(0, len(df_ecg['ecgvalues']))
ax.scatter(component_index, df_ecg['ecgvalues'], color = 'white', marker = 'o', s = 5)
ax.set_xlabel('Sample Number')
ax.set_ylabel('Normalized Voltage')
ax.set_title('ECG Signal With Missing Values')

---

Extract the indices corresponding to the missing and known ECG values. For simplicity, assume that the full ECG signal is a 6-vector x with samples 3, 5, and 6 missing as shown below:

$$\begin{align*}
\mathbf{x} &= \begin{bmatrix}x_1\\x_2\\{\color{yellow}{x_3}}\\x_4\\{\color{yellow}{x_5}}\\{\color{yellow}{x_6}}\end{bmatrix} = \begin{bmatrix}x_1\\x_2\\{\color{yellow}{v_1}}\\x_4\\{\color{yellow}{v_2}}\\{\color{yellow}{v_3}}\end{bmatrix}.
\end{align*}$$

---

In [ ]:
# Extract the indices corresponding to the missing ECG values
indices_unknown = ?

# Extract the indices corresponding to the known ECG values
indices_known = ?

---

Build a vector of known ecg_signal values. Using the simple ECG signal example, we get:

$$\mathbf{x}_\text{known} = \begin{bmatrix}x_1\\x_2\\x_4\end{bmatrix}.$$

---

In [ ]:
## Build a vector of known ecg_signal values
x_known = ?
print(x_known)

---

Build the sampling matrices $\mathbf{S}_1$ and $\mathbf{S}_2$ such that

$$\begin{align*}
\underbrace{\begin{bmatrix}?&?&?\\?&?&?\\?&?&?\\?&?&?\\?&?&?\\?&?&?\end{bmatrix}}_{\mathbf{S}_1}\begin{bmatrix}x_1\\x_2\\x_4\end{bmatrix}+\underbrace{\begin{bmatrix}?&?&?\\?&?&?\\?&?&?\\?&?&?\\?&?&?\\?&?&?\end{bmatrix}}_{\mathbf{S}_2}\begin{bmatrix}{\color{yellow}{v_1}}\\{\color{yellow}{v_2}}\\{\color{yellow}{v_3}}\end{bmatrix}=\begin{bmatrix}x_1\\x_2\\{\color{yellow}{v_1}}\\x_4\\{\color{yellow}{v_2}}\\{\color{yellow}{v_3}}\end{bmatrix}.
\end{align*}$$

---

In [ ]:
I = ?
S1 = ?
S2 = ?

---

Build the second-difference matrix $\mathbf{D}$ which for the simple ECG example is

$$\mathbf{D} = \begin{bmatrix}1&-2&1&0&0&0\\0&1&-2&1&0&0\\0&0&1&-2&1&0\\0&0&0&1&-2&1\end{bmatrix}.$$

Note that the shape of $\mathbf{D}$ is $(6-2)\times6 = 4\times6.$

We achieve this by building a $6\times6$-matrix as follows and removing the 1st and last rows:

$$\mathbf{D} = \begin{bmatrix}\color{cyan}{-2}&\color{pink}1&0&0&0&0\\\color{pink}1&\color{cyan}{-2}&\color{pink}1&0&0&0\\0&\color{pink}1&\color{cyan}{-2}&\color{pink}1&0&0\\0&0&\color{pink}1&\color{cyan}{-2}&\color{pink}1&0\\0&0&0&\color{pink}1&\color{cyan}{-2}&\color{pink}1\\0&0&0&0&\color{pink}1&\color{cyan}{-2}\end{bmatrix}.$$


---

In [ ]:
## Build the second-difference matrix D for the simple ECG example
a = np.array([1, 1, 1, 1, 1])
b = np.array([-2, -2, -2, -2, -2, 2])
D = ?
print(D)
print('D after removing 1st and last rows')
D = ?
print(D)

---

Using the simple ECG example, we will visualize the effect of multiplying a signal vector by the matrix $\mathbf{D}.$

---

In [ ]:
## Using the simple ECG example, we will visualize the effect of multiplying
## a signal vector by the matrix D

a = np.array([10, 10, 10, 10, 10, 10]) # a constant signal
b = np.array([10, 20, 30, 40, 50, 60]) # an increasing signal
c = np.array([10, -10, 10, -10, 10, -10]) # a rapidly changing signal
d = np.array([10, 0, 10, 20, 40, 00]) # a not-so-rapidly changing signal

fig, axes = plt.subplots(2, 2, figsize = (5, 4))
ax1, ax2, ax3, ax4 = axes.flatten()
fig.tight_layout(pad = 4.0)
ax1.plot(np.arange(len(a)), a, color = 'yellow', marker = 'o')
ax1.set_xlabel('Sample number')
ax1.set_ylabel('Signal value')
ax2.plot(np.arange(len(b)), b, color = 'cyan', marker = 'o')
ax2.set_xlabel('Sample number')
ax2.set_ylabel('Signal value')
ax3.plot(np.arange(len(c)), c, color = 'orange', marker = 'o')
ax3.set_xlabel('Sample number')
ax3.set_ylabel('Signal value')
ax4.plot(np.arange(len(d)), d, color = 'magenta', marker = 'o')
ax4.set_xlabel('Sample number')
ax4.set_ylabel('Signal value')

print(f'D times constant signal a = Da = {np.dot(D, a)} => ||Da|| = {np.linalg.norm(np.dot(D, a))}')
print(f'D times increasing signal b = Db = {np.dot(D, b)} => ||Db|| = {np.linalg.norm(np.dot(D, b))}')
print(f'D times rapidly changing signal c = Dc = {np.dot(D, c)} => ||Dc|| = {np.linalg.norm(np.dot(D, c))}')
print(f'D times not so rapidly changing signal d = Dd = {np.dot(D, d)}  => ||Dd|| = {np.linalg.norm(np.dot(D, d))}')

---

Build the second-difference matrix D for the ECG data

---

In [ ]:
## Build the second-difference matrix D for the ECG data
n = len(df_ecg) # size of the signal
a = ?
b = ?
D = ?
D = D[1:-1, :]
D.shape

---

Solving an inconsitent system of equations using the least squares approach.

Consider solving the system of equations $\mathbf{A}{\color{yellow}{\mathbf{v}}} = \mathbf{b},$ where

$$\mathbf{A} = \begin{bmatrix}1&0&0\\0&1&0\\0&0&1\\0&0&0\end{bmatrix}\ \text{and }\mathbf{b} = \begin{bmatrix}1\\2\\3\\4\end{bmatrix}.$$

Note that there is no solution to this system of equations. We refer to the system as being "inconsistent." The "best" approximation for the solution that we can come up with is

$$\hat{\color{yellow}{\mathbf{v}}} = \begin{bmatrix}1\\2\\3\end{bmatrix}.$$

This solution is best in the sense that $$\mathbf{A}\hat{\color{yellow}{\mathbf{v}}} = \begin{bmatrix}1&0&0\\0&1&0\\0&0&1\\0&0&0\end{bmatrix}\begin{bmatrix}1\\2\\3\end{bmatrix}=\begin{bmatrix}1\\2\\3\\0\end{bmatrix}$$ is close as possible to the right hand side vector $$\mathbf{b} = \begin{bmatrix}1\\2\\3\\4\end{bmatrix}.$$ We see that $$\mathbf{A}\hat{\color{yellow}{\mathbf{v}}}-\mathbf{b}=\begin{bmatrix}1\\2\\3\\0\end{bmatrix}-\begin{bmatrix}1\\2\\3\\4\end{bmatrix}=\begin{bmatrix}0\\0\\0\\-4\end{bmatrix}$$ has most components equal to zeros and therefore has a small norm (magnitude). Therefore, $\hat{\color{yellow}{\mathbf{v}}}$ is called the least squares solution because $\lVert \mathbf{A}\hat{\color{yellow}{\mathbf{v}}}-\mathbf{b}\rVert_2^2$ is the smallest possible value.  

---

In [ ]:
## Solving an inconsitent system of equations using the least squares approach
A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 0, 0]])
print(A)
b = np.array([1, 2, 3, 4])
print(b)
# Get the least squares solution
vhat = linalg.lstsq(A, b)[0]
print(vhat)

---

Reconstruct the ECG signal such that it is smooth by solving the following linear least squares problem:

$$\begin{align*}\min_{\color{yellow}{\mathbf{v}}}\left\lVert\mathbf{D}\hat{\mathbf{x}}\right\rVert_2^2 = \min_{\color{yellow}{\mathbf{v}}}\left\lVert\mathbf{D}\left(\mathbf{S}_1\mathbf{x}_\text{known} + \mathbf{S}_2{\color{yellow}{\mathbf{v}}}\right)\right\rVert_2^2 = \min_\mathbf{v}\left\lVert\underbrace{\mathbf{D}\mathbf{S}_1\mathbf{x}_\text{known}}_{=-\mathbf{b}} + \underbrace{\mathbf{D}\mathbf{S}_2}_{=\mathbf{A}}{\color{yellow}{\mathbf{v}}}\right\rVert_2^2.\end{align*}$$

---

In [ ]:
## Reconstruct the ECG signal such that it is smooth
A = ?
b = ?
vhat = ?
#print(v[0].flatten().shape)
df_ecg.loc[indices_unknown, 'ecgvalues'] = ?

In [ ]:
# Check if there are any missing values
?

In [ ]:
## Component plot of the reconstructed ecg signal
fig, ax = plt.subplots(1, 1)
component_index = np.arange(0, len(df_ecg['ecgvalues']))
ax.scatter(component_index[np.array(indices_known)], df_ecg.loc[indices_known, 'ecgvalues'], color = 'white', s = 1)
ax.scatter(component_index[np.array(indices_unknown)], df_ecg.loc[indices_unknown, 'ecgvalues'], color = 'red', s = 1)
ax.plot(component_index, df_ecg['ecgvalues'], color = 'gray', linewidth = 0.2)
ax.set_xlabel('Sample Number')
ax.set_ylabel('Normalized Voltage')
ax.set_title('ECG Signal With Missing Values Interpolated');